#**스마트폰 센서 데이터 기반 모션 분류**
# 단계2 : 기본 모델링


## 0.미션

* 데이터 전처리
    * 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리 수행
* 다양한 알고리즘으로 분류 모델 생성
    * 최소 4개 이상의 알고리즘을 적용하여 모델링 수행
    * 성능 비교
    * 각 모델의 성능을 저장하는 별도 데이터 프레임을 만들고 비교
* 옵션 : 다음 사항은 선택사항입니다. 시간이 허용하는 범위 내에서 수행하세요.
    * 상위 N개 변수를 선정하여 모델링 및 성능 비교
        * 모델링에 항상 모든 변수가 필요한 것은 아닙니다.
        * 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교하세요.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것입니다.

## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.




* 함수 생성

### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용
* 세부 요구사항
    - 전체 데이터 'data01_train.csv' 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - 데이터프레임에 대한 기본 정보를 확인합니다.( .head(), .shape 등)

#### 1) 데이터 로딩

In [3]:
data = pd.read_csv('/content/drive/MyDrive/KTaivle/3차미니프로젝트/data01_train.csv')

In [5]:
data.drop(columns = ['subject'], inplace = True)

#### 2) 기본 정보 조회

In [10]:
data.shape

(5881, 562)

In [12]:
data.describe()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
count,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,...,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000,5881.000000
mean,0.274811,-0.017799,-0.109396,-0.603138,-0.509815,-0.604058,-0.628151,-0.525944,-0.605374,-0.465490,...,0.126955,-0.305883,-0.623548,0.008524,-0.001185,0.009340,-0.007099,-0.491501,0.059299,-0.054594
std,0.067614,0.039422,0.058373,0.448807,0.501815,0.417319,0.424345,0.485115,0.413043,0.544995,...,0.249176,0.322808,0.310371,0.339730,0.447197,0.608190,0.476738,0.509069,0.297340,0.278479
min,-0.503823,-0.684893,-1.000000,-1.000000,-0.999844,-0.999667,-1.000000,-0.999419,-1.000000,-1.000000,...,-0.965725,-0.979261,-0.999765,-0.976580,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.980143
25%,0.262919,-0.024877,-0.121051,-0.992774,-0.977680,-0.980127,-0.993602,-0.977865,-0.980112,-0.936067,...,-0.021610,-0.541969,-0.845985,-0.122361,-0.294369,-0.481718,-0.373345,-0.811397,-0.018203,-0.141555
50%,0.277154,-0.017221,-0.108781,-0.943933,-0.844575,-0.856352,-0.948501,-0.849266,-0.849896,-0.878729,...,0.133887,-0.342923,-0.712677,0.010278,0.005146,0.011448,-0.000847,-0.709441,0.182893,0.003951
75%,0.288526,-0.010920,-0.098163,-0.242130,-0.034499,-0.262690,-0.291138,-0.068857,-0.268539,-0.013690,...,0.288944,-0.127371,-0.501158,0.154985,0.285030,0.499857,0.356236,-0.511330,0.248435,0.111932
max,1.000000,1.000000,1.000000,1.000000,0.916238,1.000000,1.000000,0.967664,1.000000,1.000000,...,0.946700,0.989538,0.956845,1.000000,1.000000,0.998702,0.996078,0.977344,0.478157,1.000000


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5881 entries, 0 to 5880
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), object(1)
memory usage: 25.2+ MB


## **2. 데이터 전처리**

* 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리를 수행한다.


### (1) 데이터 분할1 : x, y

* 세부 요구사항
    - x, y로 분할합니다.

In [17]:
x = data.drop(columns=['Activity'])
y = data['Activity']

### (2) 스케일링(필요시)


* 세부 요구사항
    - 스케일링을 필요로 하는 알고리즘 사용을 위해서 코드 수행
    - min-max 방식 혹은 standard 방식 중 한가지 사용.

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_scaled = scaler.fit_transform(x)

# 스케일링된 데이터를 다시 데이터프레임으로 변환
X_scaled_df = pd.DataFrame(X_scaled, columns=x.columns)

In [37]:
X_scaled_df

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.202596,0.218248,0.103371,-0.859791,-0.902765,-0.870784,-0.850437,-0.900177,-0.872323,-0.851929,...,-0.638805,-0.563399,-0.622364,-0.150183,-0.096236,0.490897,0.167590,-0.215351,0.914853,0.789202
1,-0.133922,0.031040,0.192441,-0.861051,-0.966218,-0.918744,-0.853240,-0.964650,-0.925369,-0.865856,...,-1.499709,0.210864,-0.225448,-0.210252,0.871377,-1.273317,1.633370,1.643741,-2.786559,-0.335244
2,0.057662,0.083408,0.011623,-0.879254,-0.939217,-0.934495,-0.871492,-0.940565,-0.943429,-0.875484,...,1.050790,-0.710714,-0.663013,-0.024311,-1.171478,-1.481923,0.060053,-0.671996,0.481426,0.078430
3,0.221631,-0.449954,-0.701726,0.827624,1.003249,0.637317,0.835699,1.113624,0.623374,0.869399,...,-0.305298,0.935220,0.691463,-0.776120,1.373090,1.213607,-0.138128,-0.401392,0.766375,0.596080
4,1.774887,1.316574,3.437248,1.541168,0.803509,0.516570,1.455736,0.858146,0.627111,1.926091,...,-0.565337,0.458672,0.193691,-0.155629,-1.887657,-1.618035,-1.847602,-0.419484,0.691700,0.690781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5876,0.035246,0.137250,-0.386824,-0.871068,-0.858484,-0.750756,-0.862838,-0.838814,-0.732951,-0.863805,...,-0.297168,0.000341,-0.215293,-0.127793,-0.582010,-0.659088,-1.826134,-0.826804,0.184014,0.483884
5877,-1.231257,0.789668,0.072332,1.061059,0.773429,0.662807,1.026677,0.857645,0.750803,0.998504,...,0.796719,-0.981268,-0.823815,2.099475,1.396230,1.410070,-0.919998,-0.390506,0.820325,0.037372
5878,-0.101018,-0.013921,0.033509,-0.864956,-0.956741,-0.925641,-0.858418,-0.954306,-0.934276,-0.863584,...,0.945132,-0.903795,-0.848420,0.152043,0.514190,1.110044,-0.439619,2.184300,-1.670102,-1.857557
5879,-0.917407,-0.769458,-0.198823,1.251793,1.120554,0.044809,1.243050,1.132399,0.122677,1.257556,...,-0.381451,1.557728,1.836641,0.742877,1.236535,-1.567261,-1.795268,-0.444984,0.785556,0.285555


### (3) 데이터분할2 : train, validation

* 세부 요구사항
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42)

In [22]:
from sklearn.ensemble import RandomForestClassifier

x = data.drop(columns=['Activity'])
y = data['Activity']

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(random_state=42)

rf_model.fit(x_train, y_train)

importance = rf_model.feature_importances_
names = x_train.columns

In [23]:
importance = rf_model.feature_importances_
names = x_train.columns

feature_importance_df = pd.DataFrame({'Feature': names, 'Importance': importance})

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

print("변수 중요도 내림차순:")
print(feature_importance_df)

변수 중요도 내림차순:
                              Feature  Importance
40               tGravityAcc-mean()-X    0.037706
558              angle(X,gravityMean)    0.030092
49                tGravityAcc-max()-X    0.029231
41               tGravityAcc-mean()-Y    0.025261
50                tGravityAcc-max()-Y    0.024912
..                                ...         ...
337    fBodyAcc-bandsEnergy()-57,64.2    0.000081
14                   tBodyAcc-min()-Z    0.000079
392  fBodyAccJerk-bandsEnergy()-49,64    0.000071
320    fBodyAcc-bandsEnergy()-33,40.1    0.000060
292                fBodyAcc-maxInds-Z    0.000048

[561 rows x 2 columns]


## **3. 기본 모델링**



* 세부 요구사항
    - 최소 4개 이상의 알고리즘을 적용하여 모델링을 수행한다.
    - 각 알고리즘별로 전체 변수로 모델링, 상위 N개 변수를 선택하여 모델링을 수행하고 성능 비교를 한다.
    - (옵션) 알고리즘 중 1~2개에 대해서, 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것이다.

### (1) 알고리즘1 :

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

nums = [10, 20, 50, 100, 200, 300]

for num in nums:
  x_temp = X_scaled_df.copy()
  top_n_features = feature_importance_df['Feature'].head(num).tolist()
  x_temp_train = x_temp[top_n_features]
  X_train, X_val, y_train, y_val = train_test_split(x_temp_train, y, test_size=0.2, random_state=42)

  model = RandomForestClassifier()
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)
  accuracy = accuracy_score(y_val, y_pred)
  print(f"Number of top features: {num}")
  print(f"Validation accuracy: {accuracy}")
  print("-----------------------------------")

Number of top features: 10
Validation accuracy: 0.9549702633814783
-----------------------------------
Number of top features: 20
Validation accuracy: 0.9787595581988106
-----------------------------------
Number of top features: 50
Validation accuracy: 0.9753610875106202
-----------------------------------
Number of top features: 100
Validation accuracy: 0.9830076465590484
-----------------------------------
Number of top features: 200
Validation accuracy: 0.9847068819031436
-----------------------------------
Number of top features: 300
Validation accuracy: 0.9847068819031436
-----------------------------------


### (2) 알고리즘2 :

In [72]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

for num in nums:
  x_temp = X_scaled_df.copy()
  top_n_features = feature_importance_df['Feature'].head(num).tolist()
  x_temp_train = x_temp[top_n_features]
  X_train, X_val, y_train, y_val = train_test_split(x_temp_train, y, test_size=0.2, random_state=42)

  model = SVC()
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)
  accuracy = accuracy_score(y_val, y_pred)
  print(f"Number of top features: {num}")
  print(f"Validation accuracy: {accuracy}")
  print("-----------------------------------")

Number of top features: 10
Validation accuracy: 0.8479184367034834
-----------------------------------
Number of top features: 20
Validation accuracy: 0.9464740866610025
-----------------------------------
Number of top features: 50
Validation accuracy: 0.9558198810535259
-----------------------------------
Number of top features: 100
Validation accuracy: 0.9685641461342396
-----------------------------------
Number of top features: 200
Validation accuracy: 0.9813084112149533
-----------------------------------
Number of top features: 300
Validation accuracy: 0.9796091758708582
-----------------------------------


### (3) 알고리즘3 :

In [73]:
for num in nums:
  x_temp = X_scaled_df.copy()
  top_n_features = feature_importance_df['Feature'].head(num).tolist()
  x_temp_train = x_temp[top_n_features]
  X_train, X_val, y_train, y_val = train_test_split(x_temp_train, y, test_size=0.2, random_state=42)

  model = KNeighborsClassifier()
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)
  accuracy = accuracy_score(y_val, y_pred)
  print(f"Number of top features: {num}")
  print(f"Validation accuracy: {accuracy}")
  print("-----------------------------------")

Number of top features: 10
Validation accuracy: 0.9209855564995751
-----------------------------------
Number of top features: 20
Validation accuracy: 0.9532710280373832
-----------------------------------
Number of top features: 50
Validation accuracy: 0.9447748513169074
-----------------------------------
Number of top features: 100
Validation accuracy: 0.9558198810535259
-----------------------------------
Number of top features: 200
Validation accuracy: 0.9634664401019541
-----------------------------------
Number of top features: 300
Validation accuracy: 0.9634664401019541
-----------------------------------


### (4) 알고리즘4 :

In [75]:
for num in nums:
  x_temp = X_scaled_df.copy()
  top_n_features = feature_importance_df['Feature'].head(num).tolist()
  x_temp_train = x_temp[top_n_features]
  X_train, X_val, y_train, y_val = train_test_split(x_temp_train, y, test_size=0.2, random_state=42)

  model = LogisticRegression(max_iter=1000)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)
  accuracy = accuracy_score(y_val, y_pred)
  print(f"Number of top features: {num}")
  print(f"Validation accuracy: {accuracy}")
  print("-----------------------------------")

Number of top features: 10
Validation accuracy: 0.8589634664401019
-----------------------------------
Number of top features: 20
Validation accuracy: 0.9303313508920985
-----------------------------------
Number of top features: 50
Validation accuracy: 0.9524214103653356
-----------------------------------
Number of top features: 100
Validation accuracy: 0.9711129991503823
-----------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Number of top features: 200
Validation accuracy: 0.983857264231096
-----------------------------------
Number of top features: 300
Validation accuracy: 0.9821580288870009
-----------------------------------


### (5) 알고리즘5 :

In [76]:
for num in nums:
  x_temp = X_scaled_df.copy()
  top_n_features = feature_importance_df['Feature'].head(num).tolist()
  x_temp_train = x_temp[top_n_features]
  X_train, X_val, y_train, y_val = train_test_split(x_temp_train, y, test_size=0.2, random_state=42)

  model = GradientBoostingClassifier()
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)
  accuracy = accuracy_score(y_val, y_pred)
  print(f"Number of top features: {num}")
  print(f"Validation accuracy: {accuracy}")
  print("-----------------------------------")

Number of top features: 10
Validation accuracy: 0.935429056924384
-----------------------------------
Number of top features: 20
Validation accuracy: 0.9694137638062872
-----------------------------------
Number of top features: 50
Validation accuracy: 0.9694137638062872
-----------------------------------
Number of top features: 100
Validation accuracy: 0.9821580288870009
-----------------------------------
Number of top features: 200
Validation accuracy: 0.9932030586236194
-----------------------------------
Number of top features: 300
Validation accuracy: 0.994052676295667
-----------------------------------
